Get packages

In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime

from urllib.parse import quote
import feedparser
import requests
from bs4 import BeautifulSoup
from newspaper import Article
from textblob import TextBlob

from transformers import pipeline
import torch
import numpy as np

Web scraping

In [39]:
def get_news(query, num_results):
    rss = f"https://news.google.com/rss/search?q={quote(str(query))}"
    feed = feedparser.parse(rss)
    news = feed.entries[:num_results]

    articles = []
    for entry in news:
        title = entry.title
        published = entry.published
        link = entry.link
        content = get_content(link)
        articles.append({"title": title, "published": published, "link": link, "content": content})

    return articles

def get_content(url):
    try:
        # Get content from the URL and make content operable using soup
        response = requests.get(url, timeout=10, allow_redirects=True)
        soup = BeautifulSoup(response.content, 'html.parser')

        # find all paragraph tags and headline tags <h1> and <p>
        paragraphs = soup.find_all('p')

        # Join all paragraphs text
        text = " ".join(p.get_text() for p in paragraphs).strip()

        return text
    except requests.RequestException:
        return "content not available due to request error"


       

Get Sentiment

In [19]:
pipe = pipeline("text-classification", model="ProsusAI/finbert", return_all_scores=True)
def sentiment_analysis(text):
    # Skip if input is not a string
    if not isinstance(text, str):
        return {"error": "Invalid content"}
    
    results = pipe(text)
    sentiments = {results['label']: results['score']}
    return sentiments
    
       

Device set to use cpu


Define keywords

In [20]:
ticker = ""
keywords = ["earnings", "revenue", "profit", "loss"]
query_target = [ticker + keyword for keyword in keywords]

Analysis

In [40]:
articles = get_news(query_target, 10)
articles[0]
# for article in articles:
#     sentiment = sentiment_analysis(article['content'])
#     article['sentiment'] = sentiment

# for article in articles:
#     for k,v in article.items():
#         print(f"{k}: {v}")
#         print("-" * 40)
        


{'title': "Nebius Reports Bigger Q3 Net Income Loss, Announces Meta AI Deal - Investor's Business Daily",
 'published': 'Tue, 11 Nov 2025 13:18:00 GMT',
 'link': 'https://news.google.com/rss/articles/CBMiiwFBVV95cUxNLWQ5WVNLeEZKemhqU1g2cHdhQjFFeTM2cmpBVXhSLTBNWWtuNUZHazdqSE9MZjkzaTVCZE50MzRMNm9FekFvMnVVZkp3RkEtUWxSeGw2TWZ4U2syRWVuOFA4Tl9FazJGZ2FWM2lvSzBIUkhrclZoNlF0UXNScERRcEhBWU5qTEMxeEhz?oc=5',
 'content': ''}